## Trajectory linking: example for 2001 

**Notes**


*last updated 03 Mar 2020*

TODO 
- when applying for all years: check whether all files have the same number of timesteps 
- check whether trajectory linking must be on annual basis or whether it works for 15 years continuously 
- plot example trac, evolution step by step 

DONE 
 
- cold core criteria (remember to remove nan cells which are the not linked features)
- heavy rain core criteria 


In [18]:
import tobac 
import glob 
import numpy as np
import pandas as pd 
import os 
                                                                                                        
## Recombination of feature dataframes (update framenumbers)                                            
                                                                                                        
savedir= '/media/juli/Data/projects/data/satellite_data/ncep/ctt/Save/tbbtracking'                      
                                                                                                        
# read in HDF5 files with saved features                                                                
file_list= glob.glob(savedir  + '/Features_cells_2001??.h5')                                            
file_list.sort()                                                                                        
print('nr. of monthly feature files:', len(file_list))                                                  
                                                                                                        


nr. of monthly feature files: 12


In [86]:
# get brightness temp file 
from netCDF4 import Dataset
file = '/media/juli/Data/projects/data/satellite_data/ncep/ctt/2001/merg_200106.nc4'
ds= Dataset(file)
tbb = np.array(ds['Tb']) 

## Recombination of features 

In [81]:
from netCDF4 import Dataset    
    
i = 0                                                                                                   
frames = 0         

for file in file_list:                                                                                  
    if i == 0:                                                                                          
        Features = pd.read_hdf(file, 'table')                                                           
        # read in data mask with segments for tracked cells                                             
        date= file[len(file)-9: len(file)-3]                                                            
        ds = Dataset(savedir+ '/Mask_Segmentation_'+ date +'.nc')                                   
        mask = np.array(ds['segmentation_mask'])                                                        
        # update total nr of frames                                                                     
        frames += np.shape(mask)[0] -1                                                                  
        i = 1                                                                                           
        print('file for: ',date, 'rows: ',Features.shape[0], 'frames: ', frames)                        
                                                                                                        
    features = pd.read_hdf(file, 'table')                                                               
    # update frame number and make sure they are sequential                                             
    features['frame_new'] = features['frame']  + frames                                                     
    # append dataframes                                                                                 
    Features = Features.append(features, ignore_index=True)                                             
    # read in data mask with segments for tracked cells                                                 
    date= file[len(file)-9: len(file)-3]                                                                
    ds = Dataset(savedir+ '/Mask_Segmentation_'+date+'.nc')                                       
    mask = np.array(ds['segmentation_mask'])                                                            
    #update total nr of frames                                                                          
    frames += np.shape(mask)[0]                                                                         
    print('file for: ',date, 'rows: ',features.shape[0], 'frames: ', frames)                            
                                                                                                                                                     

file for:  200101 rows:  7162 frames:  1487


/home/juli/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


file for:  200101 rows:  7162 frames:  2975
file for:  200102 rows:  6060 frames:  4319
file for:  200103 rows:  9957 frames:  5807
file for:  200104 rows:  12440 frames:  7247
file for:  200105 rows:  13974 frames:  8735
file for:  200106 rows:  24025 frames:  10175
file for:  200107 rows:  26147 frames:  11663
file for:  200108 rows:  23214 frames:  13151
file for:  200109 rows:  13002 frames:  14591
file for:  200110 rows:  10197 frames:  16079
file for:  200111 rows:  4032 frames:  17519
file for:  200112 rows:  7630 frames:  19007


In [83]:
## Tracking                                                                                             
# Dictionary containing keyword arguments for the linking step:                                         
parameters_linking={}          
dt = 1800
dxy = 14126.0
parameters_linking['adaptive_stop']=0.2                                                                 
parameters_linking['adaptive_step']=0.95                                                                
parameters_linking['extrapolate']=0                                                                     
parameters_linking['order']=1                                                                           
parameters_linking['subnetwork_size']= 5000 # maximum size of subnetwork used for linking               
parameters_linking['memory']= 0                                                                          
parameters_linking['time_cell_min']= 6*dt                                                              
parameters_linking['method_linking']='predict'                                                          
#parameters_linking['method_detection']='threshold'                                                     
parameters_linking['v_max']= 10                                                                         
#parameters_linking['d_min']=2000                                                                       
parameters_linking['d_min']=4*dxy # four times the grid spacing ?                                       
            

In [87]:
## Perform trjactory linking with trackpy   

tracks=tobac.linking_trackpy(Features,tbb, dt=dt,dxy=dxy,**parameters_linking)                         
track.to_hdf(os.path.join(savedir,'Tracks_tbb_2001.h5'),'table')                                          
                                                                                                        
# remove nan values 
tracks = tracks[tracks.cell >= 0]                      

Frame 1487: 67 trajectories present.


/home/juli/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['time', 'timestr']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [89]:

print(np.unique(tracks.cell.values).shape[0], '  unique cloud cells in year 2001. Features:', tracks.shape[0])

5645   unique values in year 2001. Features: 59666


## Cold core criteria 

In [93]:
# once during the lifetime a cold core of <= 200 K needs to be present 
######
for i in np.unique(tracks.cell.values):
    subset = tracks[tracks.cell == i]
    if subset[subset.threshold_value <=200].shape[0] == 0:
        tracks.drop(tracks.loc[tracks['cell']== i].index, inplace=True)
 

/home/juli/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [16]:

tracks.to_hdf(os.path.join(savedir,'Tracks_tbb_2001_cold_core.h5'),'table')  


(8133, 15)

## Read in track example, filtered by cold core 

In [17]:
# get tracks with cold core 

Tracks = pd.read_hdf(savedir + '/Tracks_tbb_2001.h5', 'table')
tracks = pd.read_hdf(savedir + '/Tracks_tbb_2001_cold_core.h5', 'table')
print('original tracks reduced by', 100 - (tracks.shape[0]/ Tracks.shape[0]*100), '%')

original tracks reduced by 95.07096883674137 %


In [5]:
# remove nan values for cells 
tracks= tracks[tracks.cell> 0]
tracks.shape

(8637, 14)

In [24]:
precip.shape

(3025, 601, 351)

In [20]:
import xarray as xr 
## test with one file 

month = '06'
year = 2000
maskfile = '/media/juli/Data/projects/data/satellite_data/ncep/ctt/Save/tbbtracking/Mask_Segmentation_'+str(year) + str(month) + '.nc'
precipfile = '/media/juli/Elements/gpm_v06/'+str(year)+'/gpm_imerg_'+ str(year)+str(month)+'_monthly.nc4'
        
mask = xr.open_dataarray(maskfile)
precip = xr.open_dataarray(precipfile)

seg= mask[3,:,:]
prec= precip[3,:,:]

# Heavy rain core filter 

In [15]:
tracks = pd.read_hdf(savedir + '/Tracks_tbb_2001_heavyraincores.h5', 'table')
tracks.shape

(8133, 15)

In [8]:
tracks.sort_values(by = 'time')
np.unique(tracks.cell.values).shape

(664,)

In [13]:
tracks.shape

(8133, 15)

In [12]:

import xarray as xr 
import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

removed = 0 

# loop through cells 
for cell in np.unique(tracks.cell.values):
    subset = tracks[tracks.cell == cell]
    print('checking heavy rain cores for cell:', cell, subset.shape)
    precipitation_flag = 0 
    # loop through timesteps of features for specific cell 
    for f in subset.frame.values: 
        # idx is the timestep index for respective timestep or mask file 
        idx = f
    
        # open corresponding precip and mask file 
        year = subset.time.values[0].year 
        month = subset.time.values[0].month
        if len(str(month))== 1: 
            month= '0' + str(month)
            
        # check whether precip is in area of segmentation mask, where segmentation mask== feature number 
        maskfile = '/media/juli/Data/projects/data/satellite_data/ncep/ctt/Save/tbbtracking/Mask_Segmentation_'+str(year) + str(month) + '.nc'
        precipfile = '/media/juli/Elements/gpm_v06/'+str(year)+'/gpm_imerg_'+ str(year)+str(month)+'_monthly.nc4'
        
        mask = xr.open_dataarray(maskfile)
        precip = xr.open_dataarray(precipfile)
        ## ! check whether corresponding file exist and whether it has the same shape
    
        # get right timestep frames 
        seg= mask[idx,:,:]
        prec = precip[idx,:,:]
        
        # get feature ID for frame 
        featureid= subset.feature[subset.frame== f].values[0]
        
        # create mask as coordinates 
        seg_mask = seg.where(seg == featureid)
        seg_mask.coords['mask'] = (('lat', 'lon'), seg_mask)
        # apply mask on precip data to extract precip values for feature in cell 
        precip_values = prec.T.where(seg_mask.coords['mask'].values > 1)
        arr= precip_values.values.flatten()
        values = arr[~np.isnan(arr)]
          
        values = arr[~np.isnan(arr)] # values contains the amount of grid cells with precip > 5mm/hr
        rain_features = values[values > 10].shape[0]
        if rain_features >= 5: 
            precipitation_flag += rain_features

    if precipitation_flag ==  0: 
        # remove corresponding cell from track dataframe 
        tracks = tracks.drop(tracks[tracks.cell == cell].index)
        print(cell, ' removed.')
        removed += 1 
        
    else:
        print('heavy rain core present in:  ', cell, rain_features)
    
tracks.to_hdf(os.path.join(savedir,'Tracks_tbb_2001_heavyraincores.h5'),'table')    

checking heavy rain cores for cell: 27.0 (30, 15)
heavy rain core present in:   27.0 1
checking heavy rain cores for cell: 37.0 (11, 15)
heavy rain core present in:   37.0 13
checking heavy rain cores for cell: 68.0 (7, 15)
heavy rain core present in:   68.0 17
checking heavy rain cores for cell: 145.0 (7, 15)
heavy rain core present in:   145.0 19
checking heavy rain cores for cell: 146.0 (10, 15)
heavy rain core present in:   146.0 154
checking heavy rain cores for cell: 259.0 (14, 15)
heavy rain core present in:   259.0 1
checking heavy rain cores for cell: 286.0 (11, 15)
heavy rain core present in:   286.0 58
checking heavy rain cores for cell: 488.0 (13, 15)
heavy rain core present in:   488.0 0
checking heavy rain cores for cell: 583.0 (10, 15)
heavy rain core present in:   583.0 7
checking heavy rain cores for cell: 593.0 (28, 15)
heavy rain core present in:   593.0 0
checking heavy rain cores for cell: 705.0 (8, 15)
heavy rain core present in:   705.0 10
checking heavy rain cor

heavy rain core present in:   10090.0 36
checking heavy rain cores for cell: 10193.0 (16, 15)
10193.0  removed.
checking heavy rain cores for cell: 10222.0 (28, 15)
heavy rain core present in:   10222.0 18
checking heavy rain cores for cell: 10439.0 (25, 15)
heavy rain core present in:   10439.0 2
checking heavy rain cores for cell: 10471.0 (12, 15)
heavy rain core present in:   10471.0 2
checking heavy rain cores for cell: 10546.0 (10, 15)
heavy rain core present in:   10546.0 56
checking heavy rain cores for cell: 10891.0 (7, 15)
heavy rain core present in:   10891.0 0
checking heavy rain cores for cell: 10943.0 (7, 15)
heavy rain core present in:   10943.0 52
checking heavy rain cores for cell: 11256.0 (8, 15)
heavy rain core present in:   11256.0 32
checking heavy rain cores for cell: 11271.0 (7, 15)
heavy rain core present in:   11271.0 5
checking heavy rain cores for cell: 11347.0 (10, 15)
heavy rain core present in:   11347.0 0
checking heavy rain cores for cell: 11414.0 (9, 15)

heavy rain core present in:   16885.0 203
checking heavy rain cores for cell: 16944.0 (15, 15)
heavy rain core present in:   16944.0 0
checking heavy rain cores for cell: 17061.0 (9, 15)
heavy rain core present in:   17061.0 165
checking heavy rain cores for cell: 17185.0 (13, 15)
heavy rain core present in:   17185.0 67
checking heavy rain cores for cell: 17278.0 (14, 15)
heavy rain core present in:   17278.0 0
checking heavy rain cores for cell: 17282.0 (7, 15)
heavy rain core present in:   17282.0 288
checking heavy rain cores for cell: 17295.0 (9, 15)
heavy rain core present in:   17295.0 1
checking heavy rain cores for cell: 17313.0 (12, 15)
17313.0  removed.
checking heavy rain cores for cell: 17378.0 (8, 15)
heavy rain core present in:   17378.0 109
checking heavy rain cores for cell: 17386.0 (12, 15)
heavy rain core present in:   17386.0 1
checking heavy rain cores for cell: 17394.0 (14, 15)
heavy rain core present in:   17394.0 44
checking heavy rain cores for cell: 17464.0 (8

heavy rain core present in:   21650.0 0
checking heavy rain cores for cell: 21655.0 (8, 15)
heavy rain core present in:   21655.0 0
checking heavy rain cores for cell: 21692.0 (17, 15)
heavy rain core present in:   21692.0 0
checking heavy rain cores for cell: 21734.0 (26, 15)
heavy rain core present in:   21734.0 178
checking heavy rain cores for cell: 21885.0 (13, 15)
heavy rain core present in:   21885.0 13
checking heavy rain cores for cell: 21983.0 (16, 15)
heavy rain core present in:   21983.0 55
checking heavy rain cores for cell: 22049.0 (8, 15)
heavy rain core present in:   22049.0 129
checking heavy rain cores for cell: 22062.0 (46, 15)
heavy rain core present in:   22062.0 162
checking heavy rain cores for cell: 22093.0 (27, 15)
heavy rain core present in:   22093.0 159
checking heavy rain cores for cell: 22110.0 (12, 15)
heavy rain core present in:   22110.0 0
checking heavy rain cores for cell: 22112.0 (7, 15)
heavy rain core present in:   22112.0 190
checking heavy rain c

heavy rain core present in:   26824.0 104
checking heavy rain cores for cell: 26875.0 (13, 15)
heavy rain core present in:   26875.0 55
checking heavy rain cores for cell: 26909.0 (10, 15)
heavy rain core present in:   26909.0 0
checking heavy rain cores for cell: 26952.0 (25, 15)
heavy rain core present in:   26952.0 0
checking heavy rain cores for cell: 27148.0 (10, 15)
heavy rain core present in:   27148.0 370
checking heavy rain cores for cell: 27153.0 (12, 15)
heavy rain core present in:   27153.0 0
checking heavy rain cores for cell: 27324.0 (13, 15)
heavy rain core present in:   27324.0 18
checking heavy rain cores for cell: 27589.0 (27, 15)
heavy rain core present in:   27589.0 72
checking heavy rain cores for cell: 27748.0 (9, 15)
heavy rain core present in:   27748.0 0
checking heavy rain cores for cell: 27921.0 (7, 15)
heavy rain core present in:   27921.0 1
checking heavy rain cores for cell: 27927.0 (7, 15)
heavy rain core present in:   27927.0 81
checking heavy rain cores

heavy rain core present in:   40718.0 0
checking heavy rain cores for cell: 40828.0 (13, 15)
heavy rain core present in:   40828.0 47
checking heavy rain cores for cell: 40829.0 (15, 15)
heavy rain core present in:   40829.0 0
checking heavy rain cores for cell: 40938.0 (15, 15)
heavy rain core present in:   40938.0 43
checking heavy rain cores for cell: 41644.0 (8, 15)
heavy rain core present in:   41644.0 154
checking heavy rain cores for cell: 41736.0 (16, 15)
heavy rain core present in:   41736.0 63
checking heavy rain cores for cell: 41737.0 (8, 15)
heavy rain core present in:   41737.0 40
checking heavy rain cores for cell: 41827.0 (8, 15)
heavy rain core present in:   41827.0 8
checking heavy rain cores for cell: 41834.0 (8, 15)
heavy rain core present in:   41834.0 118
checking heavy rain cores for cell: 42019.0 (11, 15)
heavy rain core present in:   42019.0 9
checking heavy rain cores for cell: 42026.0 (15, 15)
heavy rain core present in:   42026.0 0
checking heavy rain cores 

heavy rain core present in:   50929.0 0
checking heavy rain cores for cell: 51015.0 (11, 15)
heavy rain core present in:   51015.0 1
checking heavy rain cores for cell: 51016.0 (11, 15)
heavy rain core present in:   51016.0 2
checking heavy rain cores for cell: 51095.0 (8, 15)
heavy rain core present in:   51095.0 6
checking heavy rain cores for cell: 51186.0 (14, 15)
heavy rain core present in:   51186.0 0
checking heavy rain cores for cell: 51248.0 (8, 15)
heavy rain core present in:   51248.0 16
checking heavy rain cores for cell: 51392.0 (40, 15)
heavy rain core present in:   51392.0 5
checking heavy rain cores for cell: 51623.0 (13, 15)
heavy rain core present in:   51623.0 0
checking heavy rain cores for cell: 51775.0 (25, 15)
heavy rain core present in:   51775.0 130
checking heavy rain cores for cell: 51806.0 (8, 15)
heavy rain core present in:   51806.0 10
checking heavy rain cores for cell: 51879.0 (10, 15)
heavy rain core present in:   51879.0 0
checking heavy rain cores for

heavy rain core present in:   59232.0 9
checking heavy rain cores for cell: 59316.0 (8, 15)
heavy rain core present in:   59316.0 0
checking heavy rain cores for cell: 59322.0 (12, 15)
heavy rain core present in:   59322.0 96
checking heavy rain cores for cell: 59384.0 (11, 15)
heavy rain core present in:   59384.0 1
checking heavy rain cores for cell: 59447.0 (15, 15)
heavy rain core present in:   59447.0 442
checking heavy rain cores for cell: 59523.0 (47, 15)
59523.0  removed.
checking heavy rain cores for cell: 59531.0 (7, 15)
heavy rain core present in:   59531.0 35
checking heavy rain cores for cell: 59668.0 (26, 15)
heavy rain core present in:   59668.0 0
checking heavy rain cores for cell: 59770.0 (7, 15)
heavy rain core present in:   59770.0 253
checking heavy rain cores for cell: 59865.0 (10, 15)
heavy rain core present in:   59865.0 17
checking heavy rain cores for cell: 59984.0 (7, 15)
heavy rain core present in:   59984.0 87
checking heavy rain cores for cell: 59993.0 (15,

/home/juli/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['time', 'timestr']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
